In [40]:


def parse(filename,verbose=False):
    lines = open(filename).readlines()
    D,I,S,V,F = map(int,lines[0].split())
    if verbose:
        print(filename,D,"secs",I,"intersections",S,"streets",V,"cars",F,"bonus",sep='\t')
    
    streets = []
    for line in lines[1:S+1]:
        assert(len(line.split()) == 4)
        B,E, name, L = line.split()
        B,E,L=map(int,[B,E,L])
        streets += [(B,E,name,L)]
    assert(len(streets) == S)
        
    paths = []
    for line in lines[S+1:]:
        P = int(line.split()[0])
        assert(len(line.split()) == P+1)
        paths += [line.split()[1:]]
    assert(len(paths) == V)
          
    return (D,F,streets,paths)
        
for dataset_name in "abcdef":
    parse(dataset_name+".txt",verbose=True)

None

a.txt	6	secs	4	intersections	5	streets	2	cars	1000	bonus
b.txt	5070	secs	7073	intersections	9102	streets	1000	cars	1000	bonus
c.txt	1640	secs	10000	intersections	35030	streets	1000	cars	100	bonus
d.txt	8071	secs	8000	intersections	95928	streets	1000	cars	1000	bonus
e.txt	676	secs	500	intersections	998	streets	1000	cars	500	bonus
f.txt	1992	secs	1662	intersections	10000	streets	1000	cars	500	bonus


In [43]:

import networkx as nx
import matplotlib.pyplot as plt

def make_graph(dataset_name):
    D,F,streets,paths = parse(dataset_name+".txt",verbose=False)
    G = nx.DiGraph()
    d_streets = dict()
    for street in streets:
        B,E,name,L = street
        G.add_edge(B,E,weight=0)
        d_streets[name] = street

    for path in paths:
        for street_name in path:
            B,E,name,L = d_streets[street_name]
            G[B][E]['weight'] += 1
    return G
    
nx.write_gexf(G,dataset_name+ ".gexf")
# c:\dropbox\hobbies\hashcode21\yoann_repo\data

In [53]:
for dataset_name in "abcdef":
    G = make_graph(dataset_name)
    D,F,streets,paths = parse(dataset_name+".txt",verbose=False)
    intersections = G.nodes()
    
    avg_path_len = sum([len(p) for p in paths])/len(paths)
    avg_in_deg  = sum(d for n, d in G.in_degree()) / len(intersections)
    avg_out_deg = sum(d for n, d in G.out_degree()) / len(intersections)

    print(dataset_name,avg_path_len,"avg path len","%.02f"%avg_in_deg,'avg in degree',"%.02f"%avg_out_deg,'avg out degree',sep='\t')

a	3.5	avg path len	1.25	avg in degree	1.25	avg out degree
b	26.74	avg path len	1.29	avg in degree	1.29	avg out degree
c	75.813	avg path len	3.50	avg in degree	3.50	avg out degree
d	200.0	avg path len	11.99	avg in degree	11.99	avg out degree
e	6.051	avg path len	2.00	avg in degree	2.00	avg out degree
f	136.512	avg path len	6.02	avg in degree	6.02	avg out degree


In [57]:

for dataset_name in "abcdef":
    G = make_graph(dataset_name)
    D,F,streets,paths = parse(dataset_name+".txt",verbose=False)
    max_score = 0    
    
    for path in paths:
        max_score += F + (D - len(path))
    print(dataset_name,f'{max_score:,}',"max theoretical score",sep='\t')

a	2,005	max theoretical score
b	6,043,260	max theoretical score
c	1,664,187	max theoretical score
d	8,871,000	max theoretical score
e	1,169,949	max theoretical score
f	2,355,488	max theoretical score
